In [66]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important; }</style>"))
import matplotlib.pyplot as plt

### Extacting Features of 2018 movies from Wikipedia

In [67]:
import pandas as pd 
import numpy as np 
import json 
import requests
from tmdbv3api import TMDb, Movie
from dotenv import load_dotenv
import gzip
import pickle
import os 

In [68]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)
df 

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN
...,...,...,...,...,...,...,...
265,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[163]
266,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[137]
267,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[226]
268,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",NaN,[258]


In [69]:
tmdb = TMDb()
load_dotenv()
API_KEY = os.getenv('API_KEY')
tmdb.api_key = API_KEY
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    try:
        movie_id = result[0].id
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
        data_json= response.json()
        if data_json['genres']:
            for i in range(0, len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return " ".join(genres)
        else: 
            np.NaN
    except:
        return np.NaN

In [70]:
df['Genre'] =  df['Title'].apply(get_genre)
df_2018 =  df[['Title','Cast and crew', 'Genre']]
print(df_2018.shape)
df_2018.head(2)

(270, 3)


,Title,Cast and crew,Genre
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama


In [71]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [72]:
df_2018['director_name'] = df_2018['Cast and crew'].apply(get_director)
df_2018.head(2)

,Title,Cast and crew,Genre,director_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein


In [73]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
    
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [74]:
df_2018['actor_1_name'] = df_2018['Cast and crew'].apply(get_actor1)
df_2018['actor_2_name'] = df_2018['Cast and crew'].apply(get_actor2)
df_2018['actor_3_name'] = df_2018['Cast and crew'].apply(get_actor3)
df_2018

,Title,Cast and crew,Genre,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
...,...,...,...,...,...,...,...
265,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
266,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell
267,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux
268,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell


In [75]:
new_df18 = df_2018[['director_name','actor_1_name','actor_2_name','actor_3_name','Genre','Title']].rename(columns={'Title':'movie_title', 'Genre':'genres'})
new_df18 

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Insidious: The Last Key
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,The Strange Ones
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Stratton
3,Warwick Thornton,Bryan Brown,Sam Neill,NaN,Drama History Western,Sweet Country
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,The Commuter
...,...,...,...,...,...,...
265,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Mystery Adventure Comedy Crime,Holmes & Watson
266,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Thriller Science Fiction Action Adventure,Vice
267,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Drama History,On the Basis of Sex
268,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Thriller Crime Drama Action,Destroyer


In [76]:
for col in ['actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name', 'genres']:
    new_df18[col] = new_df18[col].replace(np.nan, 'Unknown')   
new_df18['movie_title'] = new_df18['movie_title'].str.lower()

In [77]:
new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' ' + new_df18['actor_3_name'] + ' ' + new_df18['genres']
new_df18

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Horror ...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Actio...
3,Warwick Thornton,Bryan Brown,Sam Neill,Unknown,Drama History Western,sweet country,Bryan Brown Sam Neill Unknown Drama History We...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Action...
...,...,...,...,...,...,...,...
265,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Mystery Adventure Comedy Crime,holmes & watson,Will Ferrell John C. Reilly Rebecca Hall Myste...
266,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Thriller Science Fiction Action Adventure,vice,Christian Bale Amy Adams Steve Carell Thriller...
267,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Drama History,on the basis of sex,Felicity Jones Armie Hammer Justin Theroux Dra...
268,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Thriller Crime Drama Action,destroyer,Nicole Kidman Sebastian Stan Toby Kebbell Thri...


### Extacting Features of 2019 movies from Wikipedia

In [78]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)
df 

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
237,DECEMBER,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
238,DECEMBER,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[221]
239,DECEMBER,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[222]
240,DECEMBER,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[223]


In [79]:
df['Genre'] =  df['Title'].apply(get_genre)
df_2019 =  df[['Title','Cast and crew', 'Genre']]
print(df_2019.shape)
df_2019.head(2)

(242, 3)


,Title,Cast and crew,Genre
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama


In [80]:
df_2019['director_name'] = df_2019['Cast and crew'].apply(get_director)
df_2019.head(2)

,Title,Cast and crew,Genre,director_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery,Adam Robitel
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama,Jen McGowan


In [81]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].apply(get_actor1)
df_2019['actor_2_name'] = df_2019['Cast and crew'].apply(get_actor2)
df_2019['actor_3_name'] = df_2019['Cast and crew'].apply(get_actor3)
df_2019

,Title,Cast and crew,Genre,director_name,actor_1_name,actor_2_name,actor_3_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman
...,...,...,...,...,...,...,...
237,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones
238,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh
239,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong
240,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson


In [82]:
new_df19 = df_2019[['director_name','actor_1_name','actor_2_name','actor_3_name','Genre','Title']].rename(columns={'Title':'movie_title', 'Genre':'genres'})
for col in ['actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name', 'genres']:
    new_df19[col] = new_df19[col].replace(np.nan, 'Unknown')   
new_df19['movie_title'] = new_df19['movie_title'].str.lower()
new_df19['comb'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' ' + new_df19['actor_3_name'] + ' ' + new_df19['genres']
new_df19

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Horror Thriller Mystery,escape room,Taylor Russell Logan Miller Deborah Ann Woll H...
1,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Thriller Drama,rust creek,Hermione Corfield Jay Paulson Sean O'Bryan Thr...
2,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Thriller,american hangman,Donald Sutherland Vincent Kartheiser Oliver De...
3,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Drama Adventure Family,a dog's way home,Bryce Dallas Howard Edward James Olmos Alexand...
4,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Comedy Drama,the upside,Bryan Cranston Kevin Hart Nicole Kidman Comedy...
...,...,...,...,...,...,...,...
237,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Animation...
238,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Drama ...
239,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller,1917,George MacKay Dean-Charles Chapman Mark Strong...
240,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Drama...


### Extacting Features of 2020 movies from Wikipedia

In [83]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)
df['Genre'] =  df['Title'].apply(get_genre)
df_2020 =  df[['Title','Cast and crew', 'Genre']]
df_2020['director_name'] = df_2020['Cast and crew'].astype(str).apply(get_director)
df_2020['actor_1_name'] = df_2020['Cast and crew'].astype(str).apply(get_actor1)
df_2020['actor_2_name'] = df_2020['Cast and crew'].astype(str).apply(get_actor2)
df_2020['actor_3_name'] = df_2020['Cast and crew'].astype(str).apply(get_actor3)
new_df20 = df_2020[['director_name','actor_1_name','actor_2_name','actor_3_name','Genre','Title']].rename(columns={'Title':'movie_title', 'Genre':'genres'})
for col in ['actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name', 'genres']:
    new_df20[col] = new_df20[col].replace(np.nan, 'Unknown')   
new_df20['movie_title'] = new_df20['movie_title'].str.lower()
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' ' + new_df20['actor_3_name'] + ' ' + new_df20['genres']
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,nan,nan,Unknown,Unknown,Thriller Action Crime,bad boys for life,nan Unknown Unknown Thriller Action Crime
1,nan,nan,Unknown,Unknown,Action Science Fiction Comedy Family,sonic the hedgehog,nan Unknown Unknown Action Science Fiction Com...
2,nan,nan,Unknown,Unknown,Action Crime,birds of prey,nan Unknown Unknown Action Crime
3,nan,nan,Unknown,Unknown,Family Fantasy Adventure,dolittle,nan Unknown Unknown Family Fantasy Adventure
4,nan,nan,Unknown,Unknown,Thriller Science Fiction Horror,the invisible man,nan Unknown Unknown Thriller Science Fiction H...
...,...,...,...,...,...,...,...
178,Anna Mastro,Peyton Elizabeth Lee,Skylar Astin,Élodie Yung,Action Adventure Comedy Fantasy,secret society of second-born royals,Peyton Elizabeth Lee Skylar Astin Élodie Yung ...
179,Marc Carlini,Tommy Dewey,François Arnaud,Minka Kelly,Comedy Drama,she's in portland,Tommy Dewey François Arnaud Minka Kelly Comedy...
180,Cooper Karl,Madelaine Petsch,Alexander Koch,Unknown,Thriller,sightless,Madelaine Petsch Alexander Koch Unknown Thriller
181,Joe Mantello,Jim Parsons,Zachary Quinto,Matt Bomer,Drama,the boys in the band,Jim Parsons Zachary Quinto Matt Bomer Drama


### Extacting Features of 2021 movies from Wikipedia

In [84]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)
df['Genre'] =  df['Title'].apply(get_genre)
df_2021 =  df[['Title','Cast and crew', 'Genre']]
df_2021['director_name'] = df_2021['Cast and crew'].astype(str).apply(get_director)
df_2021['actor_1_name'] = df_2021['Cast and crew'].astype(str).apply(get_actor1)
df_2021['actor_2_name'] = df_2021['Cast and crew'].astype(str).apply(get_actor2)
df_2021['actor_3_name'] = df_2021['Cast and crew'].astype(str).apply(get_actor3)
new_df21 = df_2021[['director_name','actor_1_name','actor_2_name','actor_3_name','Genre','Title']].rename(columns={'Title':'movie_title', 'Genre':'genres'})
for col in ['actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name', 'genres']:
    new_df21[col] = new_df21[col].replace(np.nan, 'Unknown')   
new_df21['movie_title'] = new_df21['movie_title'].str.lower()
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' ' + new_df21['actor_3_name'] + ' ' + new_df21['genres']
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,nan,nan,Unknown,Unknown,Action Adventure Fantasy,shang-chi and the legend of the ten rings*,nan Unknown Unknown Action Adventure Fantasy
1,nan,nan,Unknown,Unknown,Science Fiction Action Adventure,venom: let there be carnage*,nan Unknown Unknown Science Fiction Action Adv...
2,nan,nan,Unknown,Unknown,Action Adventure Science Fiction,black widow,nan Unknown Unknown Action Adventure Science F...
3,nan,nan,Unknown,Unknown,Action Crime Thriller,f9,nan Unknown Unknown Action Crime Thriller
4,nan,nan,Unknown,Unknown,Science Fiction Thriller Horror,a quiet place part ii,nan Unknown Unknown Science Fiction Thriller H...
...,...,...,...,...,...,...,...
266,Stephen Chbosky,Ben Platt,Kaitlyn Dever,Amandla Stenberg,Drama,dear evan hansen,Ben Platt Kaitlyn Dever Amandla Stenberg Drama
267,"Robert Cullen, Jose Ucha",Vanessa Hudgens,Kimiko Glenn,James Marsden,Animation Family Fantasy Comedy Music Adventure,my little pony: a new generation,Vanessa Hudgens Kimiko Glenn James Marsden Ani...
268,Antoine Fuqua,Jake Gyllenhaal,Ethan Hawke,Riley Keough,Drama Thriller,the guilty,Jake Gyllenhaal Ethan Hawke Riley Keough Drama...
269,Sarah Adina Smith,Kristine Froseth,Diana Silvers,Jacqueline Bisset,Drama,birds of paradise,Kristine Froseth Diana Silvers Jacqueline Biss...


In [85]:
print(new_df18.shape)
new_df18.isna().sum()

(270, 7)


director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [86]:
print(new_df19.shape)
new_df19.isna().sum()

(242, 7)


director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [87]:
print(new_df20.shape)
new_df20.isna().sum()

(183, 7)


director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [88]:
print(new_df21.shape)
new_df21.isna().sum()

(271, 7)


director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [89]:
my_df = new_df18.append(new_df19.append(new_df20.append(new_df21, ignore_index=True), ignore_index=True), ignore_index=True)
print(my_df.shape)
my_df.head(2)

(966, 7)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Horror ...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...


In [90]:
print(my_df.shape)
print(my_df.isna().sum())

#my_df.dropna(how='any', inplace=True)

print(my_df.shape)
print(my_df.isna().sum())

(966, 7)
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64
(966, 7)
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64


In [91]:
old_df = pd.read_csv('new_data.csv')
print(old_df.shape)
print(old_df.isna().sum())

(5364, 7)
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64


In [95]:
final_df = old_df.append(my_df, ignore_index=True)
print(final_df.shape)

#for col in ['actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name', 'genres']:
#    final_df[col] = final_df[col].replace(np.nan, 'Unknown')

(6330, 7)


In [98]:
final_df.to_csv('final_data.csv', index=False)

In [99]:
with gzip.open('final_data.pickle.gz', 'wb') as f :
    pickle.dump(final_df, f)

In [100]:
with gzip.open('final_data.pickle.gz', 'rb') as f :
    final_df_pickle = pickle.load(f)

In [101]:
final_df_pickle

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi Action ...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Actio...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,Unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker Unknown Documentary
...,...,...,...,...,...,...,...
6325,Stephen Chbosky,Ben Platt,Kaitlyn Dever,Amandla Stenberg,Drama,dear evan hansen,Ben Platt Kaitlyn Dever Amandla Stenberg Drama
6326,"Robert Cullen, Jose Ucha",Vanessa Hudgens,Kimiko Glenn,James Marsden,Animation Family Fantasy Comedy Music Adventure,my little pony: a new generation,Vanessa Hudgens Kimiko Glenn James Marsden Ani...
6327,Antoine Fuqua,Jake Gyllenhaal,Ethan Hawke,Riley Keough,Drama Thriller,the guilty,Jake Gyllenhaal Ethan Hawke Riley Keough Drama...
6328,Sarah Adina Smith,Kristine Froseth,Diana Silvers,Jacqueline Bisset,Drama,birds of paradise,Kristine Froseth Diana Silvers Jacqueline Biss...
